In the top right, click "Connect". If you already see a green tick, you are connected

Paste the latitude and longitude from Google Maps into the field below

From "Runtime", click "Run all"

The outputs will be printed below

In [39]:
#@title
%%capture
!pip install pyproj 3.4.1

In [ ]:
#@title
def xy_to_osgb(easting, northing, precision = 1):
    MAJOR_LETTERS = {0: {0: 'S', 1: 'N', 2: 'H'},
                     1: {0: 'T', 1: 'O'}}
    MINOR_LETTERS = {0: {0: 'V', 1: 'Q', 2: 'L', 3: 'F', 4: 'A'},
                     1: {0: 'W', 1: 'R', 2: 'M', 3: 'G', 4: 'B'},
                     2: {0: 'X', 1: 'S', 2: 'N', 3: 'H', 4: 'C'},
                     3: {0: 'Y', 1: 'T', 2: 'O', 3: 'J', 4: 'D'},
                     4: {0: 'Z', 1: 'U', 2: 'P', 3: 'K', 4: 'E'}}
    
    if precision not in [100000, 10000, 1000, 100, 10, 1]:
        raise Exception('Precision of ' + str(precision) + ' is not supported')

    try:
        x_idx = easting // 500000
        y_idx = northing // 500000
        major_letter = MAJOR_LETTERS[x_idx][y_idx]

        macro_easting = easting % 500000
        macro_northing = northing % 500000
        macro_x_idx = macro_easting // 100000
        macro_y_idx = macro_northing // 100000
        minor_letter = MINOR_LETTERS[macro_x_idx][macro_y_idx]
    except (ValueError, IndexError, KeyError, AssertionError):
        raise Exception('Out of range')

    micro_easting = macro_easting % 100000
    micro_northing = macro_northing % 100000

    ref_x = micro_easting // precision
    ref_y = micro_northing // precision

    gridref_width = 0
    if precision == 10000:
        gridref_width = 1
    elif precision == 1000:
        gridref_width = 2
    if precision == 100:
        gridref_width = 3
    elif precision == 10:
        gridref_width = 4
    elif precision == 1:
        gridref_width = 5

    format_string = (r'%s%s %0' + str(gridref_width) + r'd %0' +
                     str(gridref_width) + r'd') if precision else r'%s%s %0'
    return format_string % (major_letter, minor_letter, ref_x, ref_y)


import pyproj
pyproj.network.set_network_enabled(True)
tg = pyproj.transformer.TransformerGroup(27700, 4326)
tg.download_grids(verbose = True)
accuracy = min([i.accuracy for i in tg.transformers if i.accuracy > 0])

latlong = "Latitude, longitude" #@param {type: "string"}
latitude, longitude = latlong.split(",")
latitude, longitude = float(latitude), float(longitude)

transformer = pyproj.Transformer.from_crs("epsg:4326", "epsg:27700", always_xy = True)
easting, northing = transformer.transform(longitude, latitude)

print(xy_to_osgb(int(easting), int(northing)))
print("https://www.google.com/maps/place/" + "{:.6f}".format(latitude) + "," + "{:.6f}".format(longitude))